In [3]:
# What about class imbalance?
from load_data import NameDataset
import json 
import torch
import matplotlib.pyplot as plt

x_data = NameDataset().x_data
y_data = NameDataset().y_data

with open("data/int_to_country.json") as file:
    int_to_country = json.load(file)


country_count = {}

for i in y_data:
    country_idx = (i == 1).nonzero(as_tuple=True)[0].item()
    country = int_to_country[str(country_idx)]
    if country not in country_count: country_count[country] = 1
    else: country_count[country] += 1

country_count = {k: v for k, v in sorted(country_count.items(), key=lambda item: item[1])}
for i, j in country_count.items():
    print(i, "\t\t", j)

othercountries 		 5
luxembourg 		 72
malta 		 98
moldova 		 108
montenegro 		 119
georgia 		 140
belarus 		 170
ukraine 		 191
belgium 		 196
slovakia 		 199
vietnam 		 200
poland 		 210
russia 		 210
slovenia 		 213
austria 		 219
czechrepublic 		 285
sweden 		 291
hungary 		 292
swiss 		 303
armenia 		 304
macedonia 		 305
azerbaijan 		 312
serbia 		 314
ireland 		 325
norway 		 361
croatia 		 367
denmark 		 371
kazakhstanuzbekistanetc 		 393
latvia 		 474
finland 		 603
kosovo 		 605
estonia 		 613
greece 		 622
portugal 		 710
bosniaandherzegovina 		 741
lithuania 		 750
israel 		 782
spain 		 872
albania 		 922
iceland 		 1141
korea 		 1257
japan 		 1266
indiasrilanka 		 1274
bulgaria 		 1291
france 		 1310
greatbritain 		 1362
romania 		 1400
germany 		 1518
eastfrisia 		 1519
turkey 		 1544
arabiapersia 		 1660
thenetherlands 		 2095
italy 		 2190
usa 		 2586
china 		 7255


In [44]:
# Load the model
import torch
rnn = torch.load("output/model.pt")

# Load the test data
from load_data import NameDataset, collate_fn

dataset = NameDataset(reduce=True)
train_set, val_set, test_set = torch.utils.data.random_split(
    dataset, lengths=[11300, 1413, 1413]
)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=16,
                                          collate_fn=collate_fn)


In [45]:
predictions = []
targets = []

# Iterate over the test loader and save the predictions
for idx, (x_test, y_test) in enumerate(test_loader):
    y_test = torch.argmax(y_test, dim=1)
    prediction = rnn(x_test)
    prediction = torch.argmax(prediction, dim=1)

    predictions.append(prediction)
    targets.append(y_test)

predictions = torch.cat(predictions)
targets = torch.cat(targets)

In [46]:
# Make a confusion matrix
import json
with open("data/int_to_country.json") as file:
    countries = json.load(file)

print(countries)
print(sum(predictions == targets) / len(predictions))

import torchmetrics
confmat = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=4).cuda()
confmat(predictions, targets)

{'0': 'china', '1': 'italy', '2': 'thenetherlands', '3': 'usa'}


tensor([[658,  31,   1,  40],
        [ 55, 131,   1,  39],
        [ 60,  73,  23,  54],
        [ 99,  67,  14,  67]], device='cuda:0')